<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/cudf_cu_cat_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11==22.10.0 cudf-cu11==22.10.0 cugraph-cu11==22.10.0 pylibraft_cu11==22.10.0 raft_dask_cu11==22.10.0 dask_cudf_cu11==22.10.0 pylibcugraph_cu11==22.10.0 pylibraft_cu11==22.10.0
!pip install -U git+https://github.com/graphistry/pygraphistry.git@cudf-final #_cat_alex
!pip install -U git+https://github.com/graphistry/cu-cat.git@cudf_cat_regpt #-dc #@0.01.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.2/435.2 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 GB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━ 1.1/1.4 GB 179.9 MB/s eta 0:00:02

In [ ]:
import cuml,cudf
cuml.__version__

In [ ]:
!echo "0.01.0" > VERSION.txt
# !cp VERSION.txt /root/.local/lib/python3.9/site-packages/cu_cat/
!cp VERSION.txt /usr/local/lib/python3.9/dist-packages/cu_cat/

In [ ]:
# from importlib import reload
# reload(cu_cat)
import cu_cat
print(cu_cat.__file__)

In [ ]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 200)


In [ ]:
import graphistry
graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='***') ## key id, secret key
graphistry.__version__


In [ ]:
import logging
logger = logging.getLogger()


### redteam

In [ ]:
df = pd.read_csv('https://gist.githubusercontent.com/silkspace/c7b50d0c03dc59f63c48d68d696958ff/raw/31d918267f86f8252d42d2e9597ba6fc03fcdac2/redteam_50k.csv', index_col=0)
red_team = pd.read_csv('https://gist.githubusercontent.com/silkspace/5cf5a94b9ac4b4ffe38904f20d93edb1/raw/888dabd86f88ea747cf9ff5f6c44725e21536465/redteam_labels.csv', index_col=0)
df['feats'] = df.src_computer + ' ' + df.dst_computer + ' ' + df.auth_type + ' ' + df.logontype
df['feats2'] = df.src_computer + ' ' + df.dst_computer
ndf = df.drop_duplicates(subset=['feats'])
tdf = pd.concat([red_team.reset_index(), ndf.reset_index()])
tdf['node'] = range(len(tdf))
tdf.RED.sum()

749.0

In [ ]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()

g = graphistry.nodes(tdf, 'node')

g5a = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='cu_cat',
            memoize=False,
             remove_node_column=False,
            engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         10092281 function calls (9932164 primitive calls) in 98.313 seconds

   Ordered by: internal time
   List reduced from 2564 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8   23.027    2.878   37.824    4.728 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1015(_multiplicative_update_h_smallfast)
        3   13.850    4.617   13.850    4.617 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
       42    9.903    0.236    9.903    0.236 {built-in method cupy_backends.cuda.libs.nvrtc.compileProgram}
43132/42900    7.746    0.000   21.748    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
    39524    7.580    0.000    9.068    0.000 /usr/local/lib/python3.9/dist-packages/pandas/core/missing.py:61(mask_missing)
        1    7.358    7.358    8.647    8.647 /usr/local/lib/python3.9/dist-packages/cuml/internals/api_decorators.py:397(inner_with_setters)

In [ ]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()

g = graphistry.nodes(cudf.from_pandas(tdf), 'node')

g5b = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='cu_cat',
            memoize=False,
             remove_node_column=False,
            engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_cudf_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         5077691 function calls (4995804 primitive calls) in 47.269 seconds

   Ordered by: internal time
   List reduced from 2434 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8   22.658    2.832   28.505    3.563 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1015(_multiplicative_update_h_smallfast)
        3   13.874    4.625   13.874    4.625 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
     5061    5.423    0.001    5.478    0.001 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
        7    0.753    0.108    2.358    0.337 /usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:1258(_count_vocab)
    63053    0.603    0.000    0.862    0.000 /usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:275(_char_ngrams)
      248    0.525    0.002    0.701    0.003 /usr/local/lib/python3.9/dist-packages/cupyx/scipy/sp

In [ ]:
!pip install dirty_cat

In [ ]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()

g = graphistry.nodes((tdf), 'node')

g5c = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='dirty_cat',
            memoize=False,
            #  remove_node_column=False,
            engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_dirty_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         28575026 function calls (28415304 primitive calls) in 103.171 seconds

   Ordered by: internal time
   List reduced from 2520 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
2872739/2872500   20.671    0.000   36.063    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     1054   16.788    0.016   45.640    0.043 /usr/local/lib/python3.9/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
        3   15.163    5.054   15.163    5.054 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
    39524    6.915    0.000    8.488    0.000 /usr/local/lib/python3.9/dist-packages/pandas/core/missing.py:61(mask_missing)
      775    4.224    0.005   12.491    0.016 /usr/local/lib/python3.9/dist-packages/dirty_cat/_gap_encoder.py:971(_multiplicative_update_w)
   272946    3.572    0.000    7.499    0.000 /usr/local/lib/python3.9/dist-packages/scipy/sparse/_sputils.py:

### ask HN

In [ ]:
# # get the data top 3000 posts on Hacker News
askHN = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)

# df = askHN.sample(1000,replace=True) # set smaller if you want to test a minibatch 
# df = df[['text','title']]
# df['title'] = df['title'].sample(frac=1).values

In [ ]:
g = graphistry.nodes(cudf.from_pandas(askHN))
pr = cProfile.Profile()
pr.enable()

g1 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
# g1a = g1.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_cudf-cu_cat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Featurizing nodes with feature_engine=cu_cat
         60952076 function calls (60762449 primitive calls) in 452.703 seconds

   Ordered by: internal time
   List reduced from 2408 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      242  169.173    0.699  372.012    1.537 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:960(_multiplicative_update_h)
  5191507   96.734    0.000  122.449    0.000 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
  1426780   59.323    0.000   92.354    0.000 {method 'dot' of 'cupy._core.core.ndarray' objects}
       22   47.089    2.140   54.028    2.456 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1015(_multiplicative_update_h_smallfast)
  5192145    7.938    0.000   16.571    0.000 cuda/cudart.pyx:10521(cudaGetDevice)
        9    7.357    0.817    7.357    0.817 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
1205039/1

In [ ]:
# g1a = g1.umap(engine='cuml')

In [ ]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()

g2 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
# g2a = g2.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_cu_cat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Featurizing nodes with feature_engine=cu_cat
         61299567 function calls (61111505 primitive calls) in 458.337 seconds

   Ordered by: internal time
   List reduced from 1937 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      242  172.102    0.711  376.939    1.558 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:960(_multiplicative_update_h)
  5289231   98.242    0.000  124.480    0.000 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
  1454728   59.521    0.000   93.157    0.000 {method 'dot' of 'cupy._core.core.ndarray' objects}
       22   47.610    2.164   54.361    2.471 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1015(_multiplicative_update_h_smallfast)
        9    8.590    0.954    8.591    0.955 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
  5289231    8.238    0.000   17.140    0.000 cuda/cudart.pyx:10521(cudaGetDevice)
1225762/1

In [ ]:
# g2a = g2.umap(engine='cuml')

In [ ]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()


g3 = g.featurize(feature_engine='dirty_cat',memoize=False,remove_node_column=False)
# g3a = g3.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_dirtycat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

Featurizing nodes with feature_engine=dirty_cat
         34692512 function calls (34600508 primitive calls) in 126.143 seconds

   Ordered by: internal time
   List reduced from 1797 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      764   36.066    0.047   73.129    0.096 /usr/local/lib/python3.9/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
4235385/4234640   33.674    0.000   43.014    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      360   10.023    0.028   27.910    0.078 /usr/local/lib/python3.9/dist-packages/dirty_cat/_gap_encoder.py:971(_multiplicative_update_w)
        9    8.605    0.956    8.606    0.956 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
    15120    3.648    0.000    3.648    0.000 {built-in method scipy.sparse._sparsetools.coo_matvec}
       21    3.588    0.171    9.318    0.444 /usr/local/lib/python3.9/dist-packages/s

In [ ]:
# g1a.plot()

In [ ]:
# g1b.plot()

### CTU-13 malware dataset

In [ ]:
edf = pd.read_csv('https://gist.githubusercontent.com/silkspace/33bde3e69ae24fee1298a66d1e00b467/raw/dc66bd6f1687270be7098f94b3929d6a055b4438/malware_bots.csv', index_col=0)
T = edf.Label.apply(lambda x: True if 'Botnet' in x else False)
bot = edf[T]
nbot = edf[~T]
print(f'Botnet abundance: {100*len(bot)/len(edf):0.2f}%')# so botnet traffic makes up a tiny fraction of total

# let's balance the dataset in a 10-1 ratio, for speed and demonstrative purposes
negs = nbot.sample(10*len(bot))
edf = pd.concat([bot, negs])  # top part of arrays are bot traffic, then all non-bot traffic
edf = edf.drop_duplicates()

# some useful indicators for later that predict Botnet as Bool and Int
Y = edf.Label.apply(lambda x: 1 if 'Botnet' in x else 0)  # np.array(T)

# Later we will use and exploit any meaning shared between the labels in a latent distribution

# add it to the dataframe
edf['bot'] = Y

# name some columns for edges and features
src = 'SrcAddr'
dst = 'DstAddr'
good_cols_with_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes', src, dst]

good_cols_without_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']

## some encoding parameters
n_topics = 20
n_topics_target = 7



Botnet abundance: 0.69%


In [ ]:
# lets umap the data
g = graphistry.edges(cudf.from_pandas(edf), src, dst)

pr = cProfile.Profile()
pr.enable()
g6b = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='cu_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_cudf-cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


         20052176 function calls (19606482 primitive calls) in 38.284 seconds

   Ordered by: internal time
   List reduced from 2729 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       24    5.078    0.212    6.392    0.266 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1015(_multiplicative_update_h_smallfast)
        9    3.645    0.405    3.645    0.405 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
      146    1.564    0.011    1.564    0.011 {method 'argsort' of 'numpy.ndarray' objects}
        3    1.559    0.520    1.559    0.520 /usr/local/lib/python3.9/dist-packages/numpy/ma/core.py:3211(__getitem__)
        1    1.332    1.332    1.332    1.332 /usr/local/lib/python3.9/dist-packages/numpy/ma/core.py:3337(__setitem__)
   104188    1.180    0.000    1.640    0.000 /usr/lib/python3.9/typing.py:1065(_get_protocol_attrs)
    13055    1.087    0.000    1.087    0.000 {method 'take' 

In [ ]:
# lets umap the data
g = graphistry.edges(edf, src, dst)

pr = cProfile.Profile()
pr.enable()
g6a = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='cu_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


         15329509 function calls (14907191 primitive calls) in 37.984 seconds

   Ordered by: internal time
   List reduced from 2456 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       24    4.860    0.202    6.066    0.253 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1015(_multiplicative_update_h_smallfast)
        9    4.839    0.538    4.839    0.538 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
      146    1.571    0.011    1.571    0.011 {method 'argsort' of 'numpy.ndarray' objects}
        3    1.551    0.517    1.551    0.517 /usr/local/lib/python3.9/dist-packages/numpy/ma/core.py:3211(__getitem__)
        1    1.360    1.360    1.360    1.360 /usr/local/lib/python3.9/dist-packages/numpy/ma/core.py:3337(__setitem__)
    13078    1.052    0.000    1.052    0.000 {method 'take' of 'numpy.ndarray' objects}
    12891    0.962    0.000    3.460    0.000 /usr/local/lib/python3.9/di

In [ ]:
### lets umap the data
g = graphistry.edges(edf, src, dst)
pr = cProfile.Profile()
pr.enable()
g6c = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='dirty_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


### 20newsgroups

In [ ]:
from sklearn.datasets import fetch_20newsgroups
# newsgroups = fetch_20newsgroups()#categories=categories)
n_samples = 1000
# n_features = 1000
# n_components = 10
# n_top_words = 20
# batch_size = 128
# init = "nndsvda"

news, _ = fetch_20newsgroups(
    shuffle=True,
    random_state=1,
    remove=("headers", "footers", "quotes"),
    return_X_y=True,
)

news = news[:n_samples]
news=pd.DataFrame(news)

In [ ]:
g = graphistry.nodes(cudf.from_pandas(news))
pr = cProfile.Profile()
pr.enable()


g8 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
g8b = g8.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_cudf-cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()

g7 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
g7a = g7.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_cu_cat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()

g9 = g.featurize(feature_engine='dirty_cat',memoize=False,remove_node_column=False)
g9c = g9.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp *.txt /content/drive/MyDrive/graphistry

### winlogs

In [ ]:
!wget -nc https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet
winlogs=pd.read_parquet('part.88.parquet')
# winlogs=pd.read_parquet('part.88 (1).parquet')
winlogs=winlogs.sample(100000,replace=False).iloc[:,:10]

# g = graphistry.nodes(winlogs)
# pr = cProfile.Profile()
# pr.enable()

# g10 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
# # g10a = g10.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

# pr.disable()
# stats = Stats(pr)
# stats.sort_stats('tottime').print_stats(20)

# with open('winlogs_featurize+umap_cu_cat_100k.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)

g = graphistry.nodes(cudf.from_pandas(winlogs))
pr = cProfile.Profile()
pr.enable()

g11 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
# g11b = g11.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize+umap_cudf-cu_cat_100k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


# g = graphistry.nodes(winlogs)
# pr = cProfile.Profile()
# pr.enable()


# g12 = g.featurize(feature_engine='dirty_cat',memoize=False,remove_node_column=False)
# # g12c = g12.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

# pr.disable()
# stats = Stats(pr)
# stats.sort_stats('tottime').print_stats(20)

# with open('winlogs_featurize+umap_dirtycat_100k.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)

File ‘part.88.parquet’ already there; not retrieving.



Featurizing nodes with feature_engine=cu_cat
         266634181 function calls (262629716 primitive calls) in 2055.556 seconds

   Ordered by: internal time
   List reduced from 2215 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1328  791.614    0.596 1738.558    1.309 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:960(_multiplicative_update_h)
 26310839  431.685    0.000  553.090    0.000 /usr/local/lib/python3.9/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
  7218934  277.486    0.000  429.702    0.000 {method 'dot' of 'cupy._core.core.ndarray' objects}
       21  237.325   11.301  237.325   11.301 /usr/local/lib/python3.9/dist-packages/numpy/lib/arraysetops.py:519(in1d)
       48   43.900    0.915   49.313    1.027 /usr/local/lib/python3.9/dist-packages/cu_cat/_gap_encoder.py:1015(_multiplicative_update_h_smallfast)
 26311936   36.514    0.000   76.378    0.000 cuda/cudart.pyx:10521(cudaGetDevice)
 26311

In [ ]:
!cp *.txt /content/drive/MyDrive/graphistry

cp: target '/content/drive/MyDrive/graphistry' is not a directory


In [ ]:
winlogs=pd.read_parquet('part.88.parquet')
winlogs=winlogs.sample(1000000,replace=False).iloc[:,:10]

# g = graphistry.nodes(winlogs)
# pr = cProfile.Profile()
# pr.enable()

# g13 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
# g13a = g13.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

# pr.disable()
# stats = Stats(pr)
# stats.sort_stats('tottime').print_stats(20)

# with open('winlogs_featurize_cu_cat_1m.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)

g = graphistry.nodes(cudf.from_pandas(winlogs))
pr = cProfile.Profile()
pr.enable()

g14 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
g14b = g14.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize_cudf-cu_cat_1m.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)
    

# g = graphistry.nodes(winlogs)
# pr = cProfile.Profile()
# pr.enable()

# g15 = g.featurize(feature_engine='cu_cat',memoize=False,remove_node_column=False)
# g15b = g15.umap(engine='cuml')#,feature_engine='cu_cat',memoize=False)


# pr.disable()
# stats = Stats(pr)
# stats.sort_stats('tottime').print_stats(20)

# with open('winlogs_featurize_dirtycat_1m.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)

In [ ]:
stopppp